실습을 시작하기 전에, 코드가 올바른 폴더에 위치해 있는지 점검해보자.

**실습 파일과 데이터셋은 반드시 '내 드라이브' 위치에 있어야 한다.**

뒤에 ratings.txt를 읽어오는데 어려움을 겪고 있다면, 파일들이 올바른 위치에 있는지 다시 한 번 점검해보자.

# **Preprocessing**

우리는 문장을 입력해주었을 때, 그 문장이 긍정문인지 부정문인지 판단해주는 AI 모델을 구축하고자 한다. AI 모델이 문장을 올바르게 분류하기 위해서는 문장 내 정보들을 인지하는 과정이 필요하다.

컴퓨터는 우리가 사용하는 언어를 있는 그대로 받아들일 수 없다. 컴퓨터가 이해할 수 있는 방식, 숫자로 변환해주어야 한다. 이와 같이 AI 모델이 주어진 데이터를 이해할 수 있도록, 데이터의 형태를 변환해주는 과정을 전처리 (Preprocessing) 라고 한다.

**본 파일에서는 문장 데이터를 전처리하는 과정에 대해 다뤄볼 것이다.**

전처리에 필요한 함수들을 구성한 뒤, 최하단의 preproc_test 함수를 통해서 테스트해볼 것이다.

실습을 시작하기에 앞서, 필요한 라이브러리들을 설치하자.

In [1]:
!pip install mxnet
!pip install sentencepiece
!pip install transformers==4.20.1
!pip install --upgrade -q pyproj

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 MB 10.8 MB/s eta 0:00:00
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.20.3
    Uninstalling graphviz-0.20.3:
      Successfully uninstalled graphviz-0.20.3
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.3/77.3 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 25.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 46.5 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.19.1
    Uninstalling tokenizers-0.19.1:
      Successfully uninstalled tokenizers-0.19.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.42.4
    Uninstalling transformers-4.42.4:
      Successfully uninstalled transformers-4.42.4


아래의 mount 함수를 통해 구글 드라이브에 접근하자.

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
!cp /content/gdrive/MyDrive/12_BERT/tokenization.py /content/

연습 문제에 필요한 라이브러리들을 import하자.

In [4]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
import numpy as np
from tqdm import tqdm, tqdm_notebook
from sklearn.model_selection import train_test_split
from keras.utils import pad_sequences
import pandas as pd
import traceback
import torch
from transformers import get_linear_schedule_with_warmup, BertForSequenceClassification, BertConfig
import random
from tqdm.notebook import tqdm
import time
# KoBERTTokenizer를 불러오자.
from tokenization import KoBertTokenizer

Pandas 라이브러리의 read_csv 함수를 활용하여 데이터셋을 읽어오자.

In [5]:
whole_dataset = pd.read_csv('/content/gdrive/MyDrive/12_BERT/ratings.txt', delimiter="\t")

읽어 온 데이터의 형태는 어떠할까?

head 함수를 활용하여 상단 5개의 data들을 출력해보고, data의 row와 column은 어떻게 구성되어 있는지 살펴보자.

In [6]:
whole_dataset.head()

,id,document,label
0,8112052,어릴때보고 지금다시봐도 재밌어요ㅋㅋ,1
1,8132799,"디자인을 배우는 학생으로, 외국디자이너와 그들이 일군 전통을 통해 발전해가는 문화산...",1
2,4655635,폴리스스토리 시리즈는 1부터 뉴까지 버릴께 하나도 없음.. 최고.,1
3,9251303,와.. 연기가 진짜 개쩔구나.. 지루할거라고 생각했는데 몰입해서 봤다.. 그래 이런...,1
4,10067386,안개 자욱한 밤하늘에 떠 있는 초승달 같은 영화.,1


In [7]:
whole_dataset['label']

0         1
1         1
2         1
3         1
4         1
         ..
199995    0
199996    0
199997    0
199998    0
199999    0
Name: label, Length: 200000, dtype: int64

In [10]:
whole_dataset[-6:]

,id,document,label
199994,4834376,개연성이 없어요.. 별루다...,0
199995,8963373,포켓 몬스터 짜가 ㅡㅡ;;,0
199996,3302770,쓰.레.기,0
199997,5458175,완전 사이코영화. 마지막은 더욱더 이 영화의질을 떨어트린다.,0
199998,6908648,왜난 재미없었지 ㅠㅠ 라따뚜이 보고나서 스머프 봐서 그런가 ㅋㅋ,0
199999,8548411,포풍저그가나가신다영차영차영차,0


# Preprocess

데이터를 살펴보니, row는 0부터 차례로 이어지며, column은 id, document, label로 구성되어 있다.

id는 데이터를 구분짓기 위한 일련 번호, document는 문장 데이터, label은 해당하는 문장이 긍정(1), 부정(0)인지 나타내는 라벨 값이다.

여기서 우리는 학습 데이터들 간에 구분할 필요가 없으므로, id 데이터는 분리하자. 그리고 column 이름으로 document보다는 sentence가 적절해보이니, 그렇게 바꿔보자.

In [27]:
def data_processing(raw_data):
    transformed_data = raw_data['label']

    # 판다스의 concat을 활용하여 'document' 데이터와 'label' 데이터를 연결해보자.
    # [1] new_array = pandas.concat([array1, array2])를 하면, array1과 array2가 물리적으로 연결된 new_array를 생성할 수 있다.
    #     이때 추가 옵션을 별도로 지정하지 않았으므로, concat을 수행하는 default 방향은 axis=0(default)이다.
    # [2] concat 시 axis 조건을 통해 array를 concat하는 방향을 직접 지정해주도록 하자.
    #     Hint: 2차원 array에서 axis=0은 세로(상/하) 방향, axis=1은 가로(좌/우) 방향이다.
    #     우리는 'document'열과 'label' 열을 가로(좌/우) 방향으로 concat해야 하므로 concat 시 axis=1이라는 조건을 설정하는 것이 적합할 것이다.
    # concat에 대한 추가적인 내용은 https://yganalyst.github.io/data_handling/Pd_12/ 를 참고해보자.
    # axis에 대한 추가적인 내용은 https://jalammar.github.io/visual-numpy/ https://lets-hci-la-ai-withme.tistory.com/15 를 참고해보자.
    ## 여기에 코드 작성
    processed_data = pd.concat([raw_data['document'], transformed_data], axis=1)

    # 데이터의 column 이름을 sentence label로 바꾸어준다.
    processed_data.columns = ['sentence', 'label']

    return processed_data

여기까지 구현한 부분을 하단의 preproc_test 함수의 첫 번째 테스트로 확인해볼 것이다. (20점)

이제 문장 데이터를 본격적으로 변환시켜볼 것이다.

아래 data_to_token_ids 함수를 아래와 같은 절차를 통해 문장 데이터를 수치적으로 변환한다.

1. Tokenizer가 문장의 시작과 끝을 인식할 수 있도록 문장 앞뒤에 CLS 토큰과 SEP 토큰을 붙인다.

2. Tokenizer의 tokenize 함수를 활용하여 문장을 여러 개의 토큰으로 나눈다.

3. Tokenizer의 convert_tokens_to_ids 함수를 활용하여, 토큰들을 대응되는 id로 변환해준다.

4. MAX_LEN의 길이에 맞춰, padding을 진행해준다. 비어있는 자리의 경우, 0이 입력된다.

In [15]:
def data_to_token_ids(tokenizer, single_sentence):
    # CLS 토큰과 SEP 토큰을 문장의 시작과 끝에 붙여보자.
    # CLS 토큰은 '[CLS]', SEP 토큰은 '[SEP]' 로 작성하자.
    # hint : 'special_token_added'는 반드시 'string' 타입, 즉 문자열 타입이어야 한다!!
    ## 여기에 코드 작성
    special_token_added = '[CLS]' + str(single_sentence) + '[SEP]'
    # tokenizer의 tokenize 함수를 활용하여 문장을 토큰화해보자.
    tokenized_sentence = tokenizer.tokenize(special_token_added)

    # tokenizer의 convert_tokens_to_ids 함수를 활용하여 생성된 토큰을 숫자 형태로 바꿔주자.
    token_ids = [tokenizer.convert_tokens_to_ids(tokenized_sentence)]

    MAX_LEN = 128
    # pad_sequences 함수를 활용하여 문장의 빈 칸에 padding을 넣어주자.
    # keras의 preprocessing.sequence 라이브러리는 pad_sequences 함수를 제공하며, 본 함수는 서로 다른 길이의 문장을 특정 길이(최대 길이)로 일치시키기 위해 truncating와 padding을 한다.
    # truncating은 최대 길이보다 긴 문장을 최대 길이에 맞게 잘라내는 것이고, padding은 최대 길이보다 짧은 문장의 남는 자리를 0으로 채우는 것을 가리킨다.
    # pad_sequence는 default 옵션은 'pre'이다. 즉 긴 문장의 앞 부분을 잘라내거나 짧은 문장의 앞에 0을 채운다.
    # 그러나 우리는 토큰 id 리스트의 '뒷'부분에 truncating 및 padding을 적용해주고자 한다.
    # truncating="post" 옵션을 통해 id 리스트의 길이가 MAX_LEN을 넘어가는 부분에 대해서는 뒷부분을 삭제할 수 있다.
    # padding="post" 옵션을 통해 토큰 id 리스트의 뒷부분에 padding을 적용할 수 있다.
    # 최대 길이는 위의 MAX_LEN으로 설정하고, dtype은 long으로 설정해주자.
    # 위 설정에 맞게 padding, truncating 값을 적절히 설정해주자.
    # https://www.tensorflow.org/api_docs/python/tf/keras/utils/pad_sequences
    ## 여기에 코드 작성
    token_ids_padded = pad_sequences(token_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")


    token_ids_flatten = token_ids_padded.flatten()
    return token_ids_flatten

완성한 data_to_token_ids 함수를 간단히 활용해보며 함수의 반환값 형태를 익혀보자.  
문장이 토큰화되고 각 토큰이 id값으로 반환되었으며, 최대 길이에 미치지 못하는 부분은 문장의 뒷부분에 0으로 padding 처리가 되었음을 확인할 수 있다.  



```
from tokenization import KoBertTokenizer
tokenizer = KoBertTokenizer.from_pretrained("monologg/kobert")

id_testing = data_to_token_ids(tokenizer, "찐배고픔이랑 가짜배고픔이랑 구분하는건 이미포기했어")

print(id_testing)

# [   2  517 7385 6312 5439 7766 7096 6022  770 6312 5439 7766 7096 6022
 1115 6416 7794 5384 3692 7728 5561 7864 6855    3    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0]

```

위와 같은 방식으로 tokenizer와 임의의 한국어 문장을 입력하여, 함수가 정확히 만들어졌는지 확인해보도록 하자.


pad_sequences 함수를 통해 padding이 이루어진 부분은 학습하는데 실질적으로 쓰이지 않는다.

그러므로, padding된 부분은 고려하지 않도록 필터링해주는 mask를 만들어보자.

여기서 mask는 padding된 부분은 지우고, 나머지 부분은 그대로 두는 기능을 한다. 고로, padding된 부분은 0, 아닌 부분은 1의 값을 나타내도록 한다.

In [16]:
def token_ids_to_mask(token_ids):

    # 한 문장에 대한 token_id 리스트를 입력으로 받는다.
    # token_id에서 0보다 큰 숫자만 유효하도록 하는 'mask' 리스트를 만들자.
    # 이 때, mask의 각 원소는 0 아니면 1의 값을 가져야 한다.
    # Hint : 각 token_id를 0 아니면 1의 값으로 바꿔주면 된다.
    # Hint : list comprehension을 활용해서 작성하면 편하다.
    # HINT : list comprehension을 활용하는 것이 가장 간결한 코드이며, 조금 어렵다면 for 문을 활용해봐도 좋다.
    # list comprehension 참고링크 :: https://www.w3schools.com/python/python_lists_comprehension.asp
    ## 여기에 코드 작성
    mask = [1 if token_id > 0 else 0 for token_id in token_ids]

    return mask

마찬가지로, 완성한 token_ids_to_mask 함수를 간단히 활용해보며 함수의 반환값 형태를 익혀보자.  
0으로 패딩처리된 부분은 0.0으로, 나머지는 1.0으로 채워진 max_length 길이의 array가 반환됨을 확인할 수 있다.

```
mask_testing = token_ids_to_mask(id_testing)
print(mask_testing)

# [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
```

위와 같은 방식으로, 앞서 생성한 id_testing을 입력하여 함수가 정확히 만들어졌는지 확인해보도록 하자.


data_to_token_ids 함수는 하나의 문장이 입력되었을 때, token_id 리스트를 출력해주는 함수이며, token_ids_to_mask 함수는 한 문장에 대한 token_id 리스트가 입력되었을 때, 하나의 mask를 생성해주는 함수다.
위에서 구성한 두 가지 함수를 활용해서, 전체 데이터를 변형하는 로직을 구성해보자.

*HINT : list comprehension을 활용하는 것이 가장 간결한 코드이며, 조금 어렵다면 for 문을 활용해봐도 좋다.*

In [22]:
# tokenize_processed_data 함수는, 앞서 정의한 함수인 data_to_token_ids와 token_ids_to_mask를 모두 활용한다.
# tokenize_processed_data 함수를 통해, 데이터를 구성하는 각 문장을 '토큰 id로 구성된, max_length 길이의 array'로 변환해주고, attention 마스크를 생성하는 작업을 한꺼번에 처리할 수 있다.
# 함수의 입력은 두 가지로, [1]tokenizer, [2] raw_data를 data_processing 함수로 전처리한 결과인 processed_dataset이다.
# 함수의 출력은 세 가지로, [1]tokenized_data (='토큰 id로 구성된 max_length 길이의 array'로 구성된 list), [2] 데이터의 labels, [3] 각 array에 대응하는 attention_masks로 구성된 list이다.

def tokenize_processed_data(tokenizer, processed_dataset):
    labels = processed_dataset['label'].to_numpy()

    # list comprehension을 활용하여 processed_dataset의 'sentence' 데이터를 id 리스트로 토큰화하자.
    # HINT : list comprehension을 활용하는 것이 가장 간결한 코드이며, 조금 어렵다면 for 문을 활용해봐도 좋다.
    # list comprehension 참고링크 :: https://www.w3schools.com/python/python_lists_comprehension.asp
    ## 여기에 코드 작성
    tokenized_data = [data_to_token_ids(tokenizer, sentence) for sentence in processed_dataset['sentence']]

    # list comprehension을 활용하여 앞서 토큰화한 id 리스트 각각을 mask로 변환하자.
    ## 여기에 코드 작성
    attention_masks = [token_ids_to_mask(token_ids) for token_ids in tokenized_data]

    return tokenized_data, labels, attention_masks

여기까지 구현한 부분을 하단의 preproc_test 함수의 두 번째 테스트로 확인해볼 것이다. (40점)

위의 함수를 통해 변환한 데이터셋을 train, validation, test용으로 나눠야 한다.

split_into_train_test 함수를 통해 train 데이터와 test 데이터로 나누고, 그렇게 나누어진 train 데이터를 split_into_train_validation 함수를 통해 train 데이터와 validation 데이터로 나눌 것이다.

sklearn.model_selection 라이브러리의 train_test_split 함수를 활용하여 아래의 두 함수를 구현해보자.

In [19]:
def split_into_train_test(whole_data, whole_label, whole_masks):
    print("length of whole_data : " + str(len(whole_data)))

    train_inputs, test_inputs, train_labels, test_labels = train_test_split(whole_data,
                                                                            whole_label,
                                                                            random_state=2022,
                                                                            test_size=0.1)
    # 위의 방식을 참조하여 mask 역시 train을 위한 mask와 test을 위한 mask로 나누자.
    # 이때 return 값을 참조하여, 우리에게 불필요한 정보는 _로 비워두자.
    # random_state와 test_size 동일하게 설정
    # train_test_split 함수 사용법 참고 링크 : https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html
    ## 여기에 코드 작성
    train_masks, test_masks = train_test_split(whole_masks,
                                               random_state=2022,
                                               test_size=0.1)

    return train_inputs, test_inputs, train_labels, test_labels, train_masks, test_masks

In [20]:
def split_into_train_validation(train_data, train_label, train_masks):
    print("length of train_data : " + str(len(train_data)))

    # split_into_train_test의 코드를 참조하여 data와 mask 각각을 train을 위한 것과 validation을 위한 것으로 나누자.
    # random_state = 2022, test_size = 0.1로 설정
    # train_test_split 함수 사용법 참고 링크 : https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html

    ## 여기에 코드 작성(data)
    train_inputs, validation_inputs, train_labels, validation_labels = train_test_split(train_data,
                                                                                        train_label,
                                                                                        random_state=2022,
                                                                                        test_size=0.1)

    ## 여기에 코드 작성(mask)
    train_masks, validation_masks = train_test_split(train_masks,
                                                      random_state=2022,
                                                      test_size=0.1)

    return train_inputs, validation_inputs, train_labels, validation_labels, train_masks, validation_masks

여기까지 구현한 부분을 하단의 preproc_test 함수의 세 번째 테스트로 확인해볼
것이다. (60점)

우리는 이렇게 나누어진 데이터들을 tensor의 형태로 변환해주어야 한다. data_to_tensor는 그러한 역할을 해주는 함수다.

torch.tensor 함수를 활용해서 inputs, labels, masks 각각을 tensor로 변환해주는 함수를 구현해보자.

In [21]:
def data_to_tensor(inputs, labels, masks):
    # 입력받은 데이터를 텐서로 변환해주는 함수
    # torch.tensor() 사용하기
    ## 여기에 코드 작성
    inputs_tensor = torch.tensor(inputs)
    labels_tensor = torch.tensor(labels)
    masks_tensor = torch.tensor(masks)
    return inputs_tensor, labels_tensor, masks_tensor

여기까지 구현한 부분을 하단의 preproc_test 함수의 네 번째 테스트로 확인해볼 것이다. (80점)

tensor로 변환한 데이터를 dataloader를 활용해서 batch 단위로 묶어줄 것이다.

batch로 데이터를 묶어주기 전에, 데이터를 어떤 순서로 뽑을 것인지 적절한 Sampler를 설정해주어야 한다. 데이터가 학습을 위한 것인지, 검증을 위한 것인지에 따라서 알맞은 Sampler를 배정해주도록 하자.

In [23]:
# 모든 DataLoader는 Sampler를 갖고 있다. Sampler는 데이터를 load 해올 때 데이터의 index를 컨트롤함으로써 어떤 데이터부터 가져올지 지정하는 기능을 한다.
# - SequentialSampler: 항상 같은 순서로, 순차적으로 데이터를 load 한다.
# - RandomSampler: 랜덤하게 데이터를 load 한다.

def tensor_to_dataloader(inputs, labels, masks, mode):
    from torch.utils.data import RandomSampler, SequentialSampler

    batch_size=32
    data = TensorDataset(inputs, masks, labels)

    if mode == "train":
        # train 모드에서는 랜덤하게 데이터를 load해오는 sampler를 사용하자.
        # 대개 mini-batch 내부 구성이 다양할수록 전체 dataset(모집단)를 잘 대표하기 때문에 주로 RandomSampler를 사용한다.
        # 참고 링크 : https://pytorch.org/docs/stable/data.html#torch.utils.data.RandomSampler
        ## 여기에 코드 작성
        data_sampler = RandomSampler(data)
    else:
        # test에는 순차적으로 데이터를 load하는 sampler을 지정하자.
        # 참고 링크 : https://pytorch.org/docs/stable/data.html#torch.utils.data.SequentialSampler
        ## 여기에 코드 작성
        data_sampler = SequentialSampler(data)

    # DataLoader 함수를 활용해서 dataloader를 선언해보자.
    # batch_size는 batch_size로 설정하고, sampler는 위에서 지정한대로 설정해주자.
    # DataLoader 사용법 : https://pytorch.org/docs/stable/data.html#torch.utils.data.DataLoader
    # sampler를 꼭 설정할것!
    ## 여기에 코드 작성
    dataloader = DataLoader(data, batch_size=batch_size, sampler=data_sampler)

    return dataloader

여기까지 구현한 부분을 하단의 preproc_test 함수의 다섯 번째 테스트로 확인해볼 것이다. (100점)

이제 전처리를 수행하는데 필요한 모든 함수들을 다 구현했다.

구현한 함수들을 모아 preproc 함수를 최종적으로 만들었다.

In [24]:
def preproc(tokenizer, whole_dataset):
    # 지금까지 만든 함수들을 이 함수 안에서 사용해서 구현하면 된다. (라이브러리를 사용하는 것이 아님)
    # whole_dataset을 전처리하자.
    processed_dataset = data_processing(whole_dataset)

    # 전처리한 전체 데이터를 토큰화하자.
    tokenized_dataset, labels, attention_masks = tokenize_processed_data(tokenizer, processed_dataset)

    # 토큰화한 전체 데이터를 train용과 test용으로 분리하자.
    train_inputs, test_inputs, train_labels, test_labels, train_masks, test_masks = split_into_train_test(tokenized_dataset, labels, attention_masks)
    # 토큰화한 train용 데이터를 train용과 validation용으로 분리하자.
    train_inputs, validation_inputs, train_labels, validation_labels, train_masks, validation_masks = split_into_train_validation(train_inputs, train_labels, train_masks)
    # train용, validation용, test용 데이터 각각을 텐서로 변환하자.
    train_inputs, train_labels, train_masks = data_to_tensor(train_inputs, train_labels, train_masks)
    validation_inputs, validation_labels, validation_masks = data_to_tensor(validation_inputs, validation_labels, validation_masks)
    test_inputs, test_labels, test_masks = data_to_tensor(test_inputs, test_labels, test_masks)

    # train용, validation용, test용 텐서를 dataloader로 변환하자.
    train_dataloader = tensor_to_dataloader(train_inputs, train_labels, train_masks, "train")
    validation_dataloader = tensor_to_dataloader(validation_inputs, validation_labels, validation_masks, "validation")
    test_dataloader = tensor_to_dataloader(test_inputs, test_labels, test_masks, "test")

    return train_dataloader, validation_dataloader, test_dataloader

함수들이 잘 적절히 잘 만들어졌는지 preproc_test 함수를 통해 측정해보자.

5개의 테스트로 구성되어 있으며, 각 테스트는 20점이다.

첫 번째 테스트는 별도의 함수는 존재하지 않는다.

In [28]:
def test2(tokenized_data):
    real_data = [2, 1706, 6664, 5729, 6983,  517, 7990, 6493, 7828, 5943, 4928, 1861, 5783, 2235,
                 6527,   54, 7227, 6160, 3010, 6559, 7828, 2846, 7095, 3394, 6946,   54, 5782, 6150,
                 3093, 6653, 7010, 5384, 3647, 2846, 6116, 4147, 6441,  517, 5693, 5693, 7828, 4768,
                 5330,  743, 5451, 6903, 4147, 7869, 6198, 4102, 2034, 7170, 7792, 4709, 7879, 7328,
                 54, 1185, 6049, 5782, 5439, 5007, 3647, 2680, 5330, 3135, 7271, 5782, 5760, 5384,
                 1861, 3100,   54, 1569, 4196, 3093, 6653, 7013, 2571,   54,    3,    0,    0,    0,
                 0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
                 0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
                 0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
                 0,    0]
    return (tokenized_data[2022] == real_data).all()


def test3(train_inputs, validation_inputs, test_masks):
    if len(train_inputs) != 162000 or len(validation_inputs) != 18000 or len(test_masks) != 20000:
        return False
    real_data = [2, 3765, 6954, 4207, 7850, 4446, 6395, 5761, 4102, 3977, 6881, 6701,   54, 2368,
                 517, 7265, 6827, 6701,   54,    3,    0,    0,    0,    0,    0,    0,    0,    0,
                 0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
                 0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
                 0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
                 0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
                 0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
                 0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
                 0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
                 0,    0]
    return (train_inputs[2022] == real_data).all()


def test4(train_inputs):
    real_input = torch.tensor([2, 3301, 6553, 6410,  517, 6193, 7591, 4179, 6141, 6255, 4244, 5439,
                               4012,  517, 6193, 7591, 1370, 5347, 5782, 5330, 2573, 6844, 7495, 1844,
                               6190, 1734, 6978, 7968, 7720, 7086,  517, 6193, 7591, 4179, 7788,  517,
                               6394, 5833, 6141, 7318, 6149, 7086, 3524, 7227, 5859, 7136, 5546, 5850,
                               2034, 7170, 7095, 1369, 5760, 1420,   55,    3,    0,    0,    0,    0,
                               0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
                               0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
                               0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
                               0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
                               0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
                               0,    0,    0,    0,    0,    0,    0,    0])

    return torch.equal(train_inputs[12345], real_input)


def test5(train_dataloader):
    real_input_ids = torch.tensor([2,  529,   54, 2860, 6295, 7640, 5371, 3594, 7837,  553,   54,  773,
                                   6383, 7095, 5037, 6645, 7837, 4501, 5957, 6629, 7288, 3714, 7207, 5357,
                                   589,   54, 2417, 5398, 6882, 3357,  631,  529, 7220,    3,    0,    0,
                                   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
                                   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
                                   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
                                   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
                                   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
                                   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
                                   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
                                   0,    0,    0,    0,    0,    0,    0,    0])

    for step, batch in enumerate(tqdm(train_dataloader)):

        if step < 1234:
            continue
        if step > 1234:
            break

        b_input_ids, b_input_mask, b_labels = batch

    return torch.equal(b_input_ids[5], real_input_ids)

def preproc_test(tokenizer, whole_dataset):

    print("================={}번째 테스트 시작===================".format(1))
    # whole_dataset을 전처리하자.
    try:
      processed_dataset = data_processing(whole_dataset)
    except:
      print(traceback.format_exc())
      return 0
    print("================={}번째 테스트 성공===================\n".format(1))


    print("================={}번째 테스트 시작===================".format(2))
    # 전처리한 전체 데이터를 토큰화하자.
    try:
      tokenized_dataset, labels, attention_masks = tokenize_processed_data(tokenizer, processed_dataset)
    except:
      print(traceback.format_exc())
      return 20
    if not test2(tokenized_dataset):
      return 20
    print("================={}번째 테스트 성공===================\n".format(2))


    print("================={}번째 테스트 시작===================".format(3))
    # 토큰화한 전체 데이터를 train용과 test용으로 분리하자.
    try:
      train_inputs, test_inputs, train_labels, test_labels, train_masks, test_masks = split_into_train_test(tokenized_dataset, labels, attention_masks)
      # 토큰화한 train용 데이터를 train용과 validation용으로 분리하자.
      train_inputs, validation_inputs, train_labels, validation_labels, train_masks, validation_masks = split_into_train_validation(train_inputs, train_labels, train_masks)
    except:
      print(traceback.format_exc())
      return 40
    if not test3(train_inputs, validation_inputs, test_masks):
      return 40
    print("================={}번째 테스트 성공===================\n".format(3))


    print("================={}번째 테스트 시작===================".format(4))
    # train용, validation용, test용 데이터 각각을 텐서로 변환하자.
    try:
      train_inputs, train_labels, train_masks = data_to_tensor(train_inputs, train_labels, train_masks)
      validation_inputs, validation_labels, validation_masks = data_to_tensor(validation_inputs, validation_labels, validation_masks)
      test_inputs, test_labels, test_masks = data_to_tensor(test_inputs, test_labels, test_masks)
    except:
      print(traceback.format_exc())
      return 60
    if not test4(train_inputs):
      return 60
    print("================={}번째 테스트 성공===================\n".format(4))


    print("================={}번째 테스트 시작===================".format(5))
    # train용, validation용, test용 텐서를 dataloader로 변환하자.
    try:
      train_dataloader = tensor_to_dataloader(train_inputs, train_labels, train_masks, "train")
      validation_dataloader = tensor_to_dataloader(validation_inputs, validation_labels, validation_masks, "validation")
      test_dataloader = tensor_to_dataloader(test_inputs, test_labels, test_masks, "test")
    except:
      print(traceback.format_exc())
      return 80
    if not test5(train_dataloader):
      return 80
    print("================={}번째 테스트 성공===================\n".format(5))


    return 100

In [29]:
# 시드 고정
seed_val = 2022
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)


tokenizer = KoBertTokenizer.from_pretrained("monologg/kobert")
score = preproc_test(tokenizer, whole_dataset)
print("현재 점수 : {}/100점".format(score))

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertTokenizer'. 
The class this function is called from is 'KoBertTokenizer'.


=================1번째 테스트 시작===================
=================1번째 테스트 성공===================

=================2번째 테스트 시작===================
=================2번째 테스트 성공===================

=================3번째 테스트 시작===================
length of whole_data : 200000
length of train_data : 180000
=================3번째 테스트 성공===================

=================4번째 테스트 시작===================


<ipython-input-21-389286a030fd>:5: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:274.)
  inputs_tensor = torch.tensor(inputs)


=================4번째 테스트 성공===================

=================5번째 테스트 시작===================


  0%|          | 0/5063 [00:00<?, ?it/s]

=================5번째 테스트 성공===================

현재 점수 : 100/100점


아래 쉘을 실행하면 테스트의 점수를 알 수 있다.

***100점이 되어야 다음 섹터로 넘어갈 수 있다.***

# Model

이번 실습의 task는 주어진 입력 문장을 긍정/부정으로 이진분류하는 모델을 만드는 것이다. 이진분류를 위한 모델을 생성하기 위해서, 우리는 사전 훈련된 BERT 모델을 가지고 온 뒤 네이버 영화 리뷰 데이터셋을 활용하여 해당 모델을 긍정/부정의 이진분류를 수행하는 모델로 학습시켜야 한다.

**본 실습에서는 사전 훈련된 BERT 모델을 가지고 와서 우리의 task에 필요한 모델을 생성하고, 모델의 학습 과정에 필요한 하이퍼 파라미터를 설정하는 과정을 다뤄볼 것이다.**

감성 이진 분류 task를 위한 BertModel을 생성하자.
- 우선, 우리가 생성할 모델을 저장할 PATH를 지정하자.
- 이후, 본 과제의 task가 무엇인지 고려하여 HuggingFace 홈페이지에서 우리에게 필요한 BertModel 형식이 무엇인지 파악하자. 그리고 해당 모델을 이용하여 model을 생성하자. (*본 과제가, 영화리뷰를 긍정과 부정의 두 가지 감정으로 분류해내는 작업임을 고려해보자.)
- 참고로 우리의 모델은 monologg의 kobert을 사전 훈련된 모델로 사용할 것이다. 사전 훈련 모델을 사용하기 위해서 from_pretrained 함수를 사용해야 하며, 함수의 파라미터에 'monologg/kobert'와 라벨 개수(num_labels)를 지정해야 한다.
- 모델을 생성하였다면, 미리 지정해둔 PATH에 해당 모델을 저장한다.

In [33]:
# [분류를 위한 BERT 모델 생성: BertModel을 초기화하는 역할]
def BertModelInitialization():
    PATH = "/content/gdrive/MyDrive/12_BERT/model.pt"

    # BertModel은 다양한 작업을 진행할 수 있도록 여러 인터페이스들을 제공한다.
    # 본 중간 미션의 task가 '영화리뷰(Sequence)를 긍정과 부정의 두 가지 감정으로 분류하기(Classification)'이다.
    # Bert에서 시퀀스를 분류하는 인터페이스로 BertForSequenceClassification가 제공되고 있다.
    # 이 외 제공되는 인터페이스 종류가 궁금하다면 HuggingFace 홈페이지에 Bert를 검색해서 찾아보자: https://huggingface.co/docs/transformers/main/en/index
    # 추가로, monologg에 의해 사전훈련된 kobert 모델을 가지고 와야 하므로 .from_pretrained('monologg/kobert')를 쓰고, 분류 라벨 수가 2개 이므로 'num_labels=2'를 추가 입력한다.

    model = BertForSequenceClassification.from_pretrained('monologg/kobert', num_labels=2)


    torch.save(model.state_dict(), PATH) # 생성한 모델을 특정 PATH에 저장하기

get_model은 생성한 BertModel을 불러와서, 그것을 우리의 디바이스에 등록하는 함수이다.

In [32]:
def get_model(device):
    PATH = "/content/gdrive/MyDrive/12_BERT/model.pt"

    model = BertForSequenceClassification.from_pretrained('monologg/kobert')

    model.load_state_dict(torch.load(PATH)) # PATH에 저장된 모델을 불러오기
    model = model.to(device) # 불러온 모델을 device에 올리기

    return model

이번에는, model의 학습 초매개변수(옵티마이저, 에포크, 훈련 스텝, 스케줄러)를 설정하는 **get_model_with_params** 함수를 정의하자.
- 우리는 학습 단계에서 최적의 매개변수를 찾아내기 위하여 옵티마이저를 사용할 수 있다. 대표적인 옵티마이저로 AdamW가 있고, 이 외에도 이하와 같이 다양한 옵티마이저들이 있다.

> Batch Gradient Descent.
Stochastic Gradient Descent.
Momentum. Nesterov Accelerated Gradient
(NAG)
Adagrad.
RMSprop.
Adam.

- 러닝 스케줄러는 학습이 이루어짐에 따라 learning_rate을 감소시키는 도구이다. 러닝 스케줄러 역시 get_linear_schedule_with_warmup 외에 다양한 종류가 있다.

우선 중간미션에서는 AdamW으로 옵티마이저 종류 및 파라미터 내용, 학습 에폭 수, 총 훈련 스텝, 러닝 스케줄러를 모두 고정할 것이다. 최종 미션에서는 이러한 초매개변수를 직접 조절하면서 모델 성능 향상을 위한 다양한 방법을 시도해볼 것이다.

In [34]:
def get_model_with_params(num_data, device):
    model = get_model(device)

    # 옵티마이저 설정하기. lr = 1e-5, eps = 1e-8으로 설정할것!
    # 옵티마이저는 AdamW를 사용할 것!
    # AdamW 사용 방법 : https://pytorch.org/docs/stable/generated/torch.optim.AdamW.html
    optimizer = optim.AdamW(model.parameters(), lr=1e-5, eps=1e-8)

    # 전체 데이터가 총 몇 번 학습되는지
    epochs = 2

    # 총 훈련 스텝
    total_steps = num_data * epochs

    # 학습이 이루어짐에 따라 learning_rate을 감소시키기 위한 스케줄러. num_training_steps를 total_steps으로 설정할것.
    # learning rate 스케쥴러안? : https://wikidocs.net/157282
    # get_linear_schedule_with_warmup를 사용할 것! : https://huggingface.co/transformers/v3.0.2/main_classes/optimizer_schedules.html#transformers.get_linear_schedule_with_warmup
    scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)

    return model, optimizer, scheduler, epochs

위에서 정의된 함수가 성공적으로 구현되어 실행되는지 확인해보자.

In [35]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
BertModelInitialization()
print(get_model_with_params(200000, device))

Downloading:   0%|          | 0.00/352M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at monologg/kobert and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at monologg/kobert and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


(BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(8002, 768, padding_idx=1)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

# Train

모델의 예측 정확도를 산출하는 함수인 **accuracy**를 정의하자. 해당 함수는 학습한 모델의 validation 점수와 test의 결과를 계산할 때 사용된다.


In [36]:
# 정확도 계산 함수
def accuracy(preds, labels):
    f_pred = np.argmax(preds, axis=1).flatten()
    f_labels = labels.flatten()
    return np.sum(f_pred == f_labels) / len(f_labels)

**잠깐 ✔ 랜덤시드 고정이란 무엇인가?**
> 학습된 모델의 결과를 동일하게 재현(Reproduction)하는 것은 여러가지 상황에서 팔요하다.  
> 모델을 돌릴 때마다 결과가 달라지지 않도록 고정하는 것이다.  

- 수상자가 되어 코드의 정합성을 검증 받게 될 경우,

- 경진대회 참가 도중 팀을 이루어 결과를 공유해야 되는 경우,

- 논문을 작성하여 그 결과를 Reproduction 해야하는 경우 등 여러 상황에서 필요하다.  

- 본 과제 역시, (1) preproc.ipynb 내 섹터 별 자동 점수 반환 및 (2) 최종 평가 과정에서 혼동을 방지하기 위하여 랜덤시드를 고정해야 한다. 주어진 2022 시드 값을 절대 수정하지 않도록 하자.

참고 자료:
https://dacon.io/codeshare/2363
https://pytorch.org/docs/stable/notes/randomness.html


In [37]:
# 재현을 위해 랜덤시드 고정
seed_val = 2022

In [38]:
# 랜덤하게 데이터를 추출하기 위한 seed 값 설정
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

학습에 활용될 데이터셋 및 토크나이저를 지정하자. 이후, 데이터셋을 전처리하여 train, validation, test 각각의 데이터로더에 입력하자.

In [39]:
from tokenization import KoBertTokenizer

# 전체 데이터를 불러오자.
whole_dataset = pd.read_csv('/content/gdrive/MyDrive/12_BERT/ratings.txt', delimiter="\t")

# KoBERTTokenizer를 불러오자.
tokenizer = KoBertTokenizer.from_pretrained("monologg/kobert")

# 방금 전에 만든 preproc 함수 사용하기
train_dataloader, validation_dataloader, test_dataloader = preproc(tokenizer, whole_dataset)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertTokenizer'. 
The class this function is called from is 'KoBertTokenizer'.


length of whole_data : 200000
length of train_data : 180000


BertModel을 생성하여 GPU 혹은 CPU에 등록하자.
- 이때 BertModelInitialization()를 실행할 경우 기존 Device에 등록된 BertModel은 초기화되니, 한 번만 실행한 이후로는 사용하지 않도록 유의하여 사용해야 한다.
- 디바이스를 설정하자.
- 본격적인 학습에 앞서 train에 대한 model, 옵티마이저, 스케줄러, 에폭을 지정하고, 모델의 그래디언트를 초기화하자.

In [40]:
# 기존 Device에 등록된 BertModel은 초기화되니, 유의하여 사용할 것.
# 한 번만 실행하고, 그 이후로는 사용하지 않도록 조심!
BertModelInitialization()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at monologg/kobert and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [41]:
# GPU 디바이스 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 방금 전에 만든 get_model_with_params 함수 사용하기
model, optimizer, scheduler, epochs = get_model_with_params(200000, device)

# 그래디언트 초기화
model.zero_grad()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at monologg/kobert and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


**train 및 validation**  
본격적으로 학습을 진행해보자. epoch 만큼 학습 loop를 반복할 것이다.
- 우리가 생성한 model에, 배치 데이터에 대한 input_ids, attention_mask, labels 변수를 입력하여 순전파를 진행할 것이다.
- 이후 역전파 과정을 통해 매개변수가 조절되며 학습이 이루어진다.
- 한 차례 학습이 이루어질 때마다 average training loss 및 validation 정확도를 출력할 것이다.
- 학습이 완료된 모델을 특정 경로(PATH)에 저장할 것이다.


In [43]:
# 에폭만큼 반복
for epoch_i in range(epochs):
    print("")
    print('========{:}번째 Epoch / 전체 {:}회 ========'.format(epoch_i + 1, epochs))
    print('훈련 중')

    total_loss = 0 # 로스 초기화
    sum_loss = 0
    model.train()  # 훈련모드로 변경

    # 데이터로더에서 배치만큼 반복하여 가져옴
    for step, batch in enumerate(tqdm(train_dataloader)):

        if step % 50 == 0:
          print("{}번째 까지의 평균 loss : {}".format(step, sum_loss/50))
          sum_loss = 0

        batch = tuple(t.to(device) for t in batch)   # 배치를 GPU에 넣음
        b_input_ids, b_input_mask, b_labels = batch  # 배치에서 데이터 추출

        # 위에서 model 함수를 정의할 때 BertForSequenceClassification를 활용하였다.
        # 여기서 BertForSequenceClassification는 input_ids, attention_mask, labels 변수를 입력받는 'forward' 함수를 내장한다.
        # forward 함수는 forward(self, input_ids, attention_mask, token_type_ids, position_ids, head_mask, inputs_embeds, labels, output_attentions, output_hidden_states)와 같은 함수 파라미터를 갖는다.
        # model 함수를 통해 forward를 수행하기 위해 우리가 입력해야 하는 변수는 input_ids, attention_mask, labels 이다.
        # 위의 코드에서 정의한 배치 데이터를 model 함수에 입력하여, 배치에 대한 Forward를 수행해보자.

        # 우리는 BertForSequenceClassification 함수를 모델로 사용하기로 하였다. 이 함수의 공식 문서를 잘 참고할것
        # 참고 링크 : https://huggingface.co/transformers/v3.0.2/model_doc/bert.html#transformers.BertForSequenceClassification
        ##여기에 코드 작성
        outputs = model(b_input_ids, attention_mask=b_input_mask, labels=b_labels)

        loss = outputs.loss # BertForSequenceClassification가 return하는 것이 어떤 것인지 꼭 확인해보자.
        total_loss += loss.item() # 총 로스 계산
        sum_loss += loss.item()

        loss.backward() # Backward 수행으로 그래디언트 계산
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0) # 그래디언트 클리핑
        optimizer.step() # 옵티마이저의 step을 이용해서 그래디언트를 통해 가중치 파라미터 업데이트
        scheduler.step() # 스케줄러로 학습률 감소. https://wikidocs.net/157282 : 이 문서에서 learning rate scheduler의 사용법 잘 숙지하기
        optimizer.zero_grad() # 그래디언트 초기화

    # 평균 로스 계산
    avg_train_loss = total_loss / len(train_dataloader)
    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))

    #### 검증 ####

    print("")
    print("검증 중")

    model.eval()

    # 변수 초기화
    eval_accuracy = 0
    nb_eval_steps = 0

    # 데이터로더에서 배치만큼 반복하여 가져옴
    for batch in validation_dataloader:
        # 배치를 GPU에 넣음
        batch = tuple(t.to(device) for t in batch)

        # 배치에서 데이터 추출
        b_input_ids, b_input_mask, b_labels = batch

        # 그래디언트 계산 안함
        with torch.no_grad():
            # Forward 수행
            outputs = model(b_input_ids, attention_mask=b_input_mask)

        # 결과 값 구함. 이 때 validation에는 label을 입력하면 안된다.
        # validation은 label 값을 입력해서 학습을 하는 것이 아니라, input을 넣어서 예측값을 얻은 후에 그것의 accuracy를 구해야 하기 때문에 label값, 즉 정답값을 알려주면 안된다
        # https://huggingface.co/transformers/v3.0.2/model_doc/bert.html#transformers.BertForSequenceClassification 이 링크를 참고하여, return 부분을 꼼꼼히 읽을 것.
        # training 할 때와 index가 다르다. 공식 문서를 꼼꼼하게 봐야 한다. 여기서 많이 실수한다.
        logits = outputs.logits

        # CPU로 데이터 이동
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()

        # 출력 로짓과 라벨을 비교하여 정확도 계산
        tmp_eval_accuracy = accuracy(logits, label_ids)
        eval_accuracy += tmp_eval_accuracy
        nb_eval_steps += 1

    print("  Accuracy: {0:.2f}".format(eval_accuracy/nb_eval_steps))

# 학습된 모델을 해당 PATH에 저장
PATH = "/content/gdrive/MyDrive/model.pt"
torch.save(model.state_dict(), PATH)

print("")
print("Training complete!")


========1번째 Epoch / 전체 2회 ========
훈련 중


  0%|          | 0/5063 [00:00<?, ?it/s]

0번째 까지의 평균 loss : 0.0
50번째 까지의 평균 loss : 0.6683310163021088
100번째 까지의 평균 loss : 0.5823659574985505
150번째 까지의 평균 loss : 0.5227385145425797
200번째 까지의 평균 loss : 0.5044405126571655
250번째 까지의 평균 loss : 0.4467291885614395
300번째 까지의 평균 loss : 0.42924466252326965
350번째 까지의 평균 loss : 0.4226857367157936
400번째 까지의 평균 loss : 0.4091164639592171
450번째 까지의 평균 loss : 0.4081563103199005
500번째 까지의 평균 loss : 0.39643811881542207
550번째 까지의 평균 loss : 0.39991283357143403
600번째 까지의 평균 loss : 0.3533689418435097
650번째 까지의 평균 loss : 0.38587633162736895
700번째 까지의 평균 loss : 0.3872189053893089
750번째 까지의 평균 loss : 0.34164385199546815
800번째 까지의 평균 loss : 0.36439061850309373
850번째 까지의 평균 loss : 0.3494753608107567
900번째 까지의 평균 loss : 0.34586771816015244
950번째 까지의 평균 loss : 0.36266241312026976
1000번째 까지의 평균 loss : 0.3524406436085701
1050번째 까지의 평균 loss : 0.32662107318639755
1100번째 까지의 평균 loss : 0.33643846660852433
1150번째 까지의 평균 loss : 0.3290845476090908
1200번째 까지의 평균 loss : 0.31890624582767485
1250번째 까지의 평균 loss : 0.3337

  0%|          | 0/5063 [00:00<?, ?it/s]

0번째 까지의 평균 loss : 0.0
50번째 까지의 평균 loss : 0.26380247235298154
100번째 까지의 평균 loss : 0.2800619985163212
150번째 까지의 평균 loss : 0.2606161317229271
200번째 까지의 평균 loss : 0.24350814767181872
250번째 까지의 평균 loss : 0.21091495431959628
300번째 까지의 평균 loss : 0.24861756980419158
350번째 까지의 평균 loss : 0.22331591188907624
400번째 까지의 평균 loss : 0.2396954445540905
450번째 까지의 평균 loss : 0.22955194100737572
500번째 까지의 평균 loss : 0.24669573679566384
550번째 까지의 평균 loss : 0.2465565560758114
600번째 까지의 평균 loss : 0.23427942849695682
650번째 까지의 평균 loss : 0.22716723680496215
700번째 까지의 평균 loss : 0.24850198581814767
750번째 까지의 평균 loss : 0.21946710407733916
800번째 까지의 평균 loss : 0.24577793672680856
850번째 까지의 평균 loss : 0.26060347437858583
900번째 까지의 평균 loss : 0.2589288590848446
950번째 까지의 평균 loss : 0.2562351222336292
1000번째 까지의 평균 loss : 0.2443066929280758
1050번째 까지의 평균 loss : 0.2306528666615486
1100번째 까지의 평균 loss : 0.25212372228503227
1150번째 까지의 평균 loss : 0.2379516462981701
1200번째 까지의 평균 loss : 0.25857344806194305
1250번째 까지의 평균 loss : 0.

**test**  
학습된 모델에 test용 데이터를 입력하여 test 결과를 출력하자.
- test용 데이터로더를 활용하여 배치 데이터에 대한 input_ids, mask를 model에 입력하여 순전파를 실행하고, 실행 결과를 outputs에 저장하자.
- 이때 test는 '학습'이 목적이 아니므로 그래디언트를 계산하지 않도록 한다.
- outputs와 실제 정답인 label_ids를 비교하여 모델의 최종 test accuracy를 확인하자. 모델이 88%의 정확도를 넘기는가?


In [44]:
print("")
print("테스트 중")

model.eval()

# 변수 초기화
eval_accuracy = 0
nb_eval_steps = 0

# 데이터로더에서 배치만큼 반복하여 가져옴
for batch in test_dataloader:
  # 배치를 GPU에 넣음
  batch = tuple(t.to(device) for t in batch)

  # 배치에서 데이터 추출
  b_input_ids, b_input_mask, b_labels = batch

  # 그래디언트 계산 안함
  with torch.no_grad():
      # Forward 수행
      outputs = model(b_input_ids, attention_mask=b_input_mask)
  # 결과 값 구함
  # Validation 때 했던 것과 같다.
  # https://huggingface.co/transformers/v3.0.2/model_doc/bert.html#transformers.BertForSequenceClassification 이 링크를 참고하여, return 부분을 꼼꼼히 읽을 것.
  # training 할 때와 index가 다르다. 공식 문서를 꼼꼼하게 봐야 한다. 여기서 많이 실수한다.
  logits = outputs.logits

  # CPU로 데이터 이동
  logits = logits.detach().cpu().numpy()
  label_ids = b_labels.to('cpu').numpy()

  # 출력 로짓과 라벨을 비교하여 정확도 계산
  tmp_eval_accuracy = accuracy(logits, label_ids)
  eval_accuracy += tmp_eval_accuracy
  nb_eval_steps += 1

print("")
print("  Accuracy: {0:.2f}".format(eval_accuracy/nb_eval_steps))


테스트 중

  Accuracy: 0.90


# main

직접 우리가 텍스트를 입력해보고 결과를 관찰해보자

In [45]:
# GPU 디바이스 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = get_model(device)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at monologg/kobert and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [46]:
while True:
  input_str = input()

  # q를 입력하면 while 루프 끝
  if input_str == "q":
    break

  # 입력받은 문장을 토큰화
  tokenizer = KoBertTokenizer.from_pretrained("monologg/kobert")
  tokenized_data = [data_to_token_ids(tokenizer, input_str)]
  attention_masks = [token_ids_to_mask(token_ids) for token_ids in tokenized_data]

  # 토큰을 텐서로 변환
  inputs_tensor = torch.tensor(tokenized_data).to(device)
  masks_tensor = torch.tensor(attention_masks).to(device)

  # 신경망의 Forward 함수 활용
  outputs = model(inputs_tensor, attention_mask=masks_tensor)

  # 확률 값에 따른 문장 분류
  logits = outputs.logits
  logits = logits.detach().cpu().numpy()
  print(logits)

  # 조건문을 활용하여 감정 이진 분류 결과를 출력하라.
  # 각 라벨이 어떤 말을 의미하는지 생각해보자.
  # 만약 "나는 당신을 정말로 사랑합니다" 라는 말을 입력하면 "당신이 입력한 문장은 긍정입니다."라는 문구가 출력되어야 하고
  # "화가 너무 난다. 진짜 짜증나네" 라는 말을 입력하면 "당신이 입력한 문장은 부정입니다."라는 문구가 출력되어야 합니다.
  ## 여기에 코드 작성. if-else 문을 사용해야함
  if logits[0][0] > logits[0][1]:
    print("당신이 입력한 문장은 부정입니다.")
  else:
    print("당신이 입력한 문장은 긍정입니다.")



사랑해


The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertTokenizer'. 
The class this function is called from is 'KoBertTokenizer'.


[[-0.12092176  0.01772921]]
당신이 입력한 문장은 긍정입니다.
미워해


The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertTokenizer'. 
The class this function is called from is 'KoBertTokenizer'.


[[-0.10484305 -0.04371025]]
당신이 입력한 문장은 긍정입니다.
파란색


...

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertTokenizer'. 
The class this function is called from is 'KoBertTokenizer'.


[[0.15366358 0.05070851]]
당신이 입력한 문장은 부정입니다.
고양이


The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertTokenizer'. 
The class this function is called from is 'KoBertTokenizer'.


[[0.11479139 0.01011359]]
당신이 입력한 문장은 부정입니다.


KeyboardInterrupt: Interrupted by user